In [3]:
import pprint
import requests
import csv
import datetime
import pandas as pd
import time
import nltk
import re
from pandas.io.json import json_normalize
from sklearn.cluster import SpectralClustering
from math import sqrt, floor

pd.set_option("display.max_columns", 50)

In [22]:
r = requests.get("https://api.nytimes.com/svc/books/v3/lists/2022-12-31/combined-print-and-e-book-fiction?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb")

In [23]:
bs = r.json()

In [24]:
for key in bs.keys():
    print(key)

status
copyright
num_results
last_modified
results


In [25]:
bs = json_normalize(bs["results"]["books"])

C:\Users\melan\AppData\Local\Temp\ipykernel_3744\577837607.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  bs = json_normalize(bs["results"]["books"])


In [26]:
bs.head()

,rank,rank_last_week,weeks_on_list,asterisk,dagger,primary_isbn10,primary_isbn13,publisher,description,price,title,author,contributor,contributor_note,book_image,book_image_width,book_image_height,amazon_product_url,age_group,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,isbns,buy_links,book_uri
0,1,1,9,0,0,1668001225,9781668001226,Atria,"In the sequel to “It Ends With Us,” Lily deals...",0.00,IT STARTS WITH US,Colleen Hoover,by Colleen Hoover,,https://storage.googleapis.com/du-prd/books/im...,322,500,https://www.amazon.com/dp/1668001225?tag=NYTBS...,,,,,,"[{'isbn10': '1668001225', 'isbn13': '978166800...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/3aa85e47-4df9-53ef-9957-a77753d3502c
1,2,2,79,0,0,1501110365,9781501110368,Atria,A battered wife raised in a violent home attem...,0.00,IT ENDS WITH US,Colleen Hoover,by Colleen Hoover,,https://storage.googleapis.com/du-prd/books/im...,319,495,http://www.amazon.com/Ends-Us-Novel-Colleen-Ho...,,,,,,"[{'isbn10': '1501110365', 'isbn13': '978150111...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/e2a3545e-e9cb-5828-9d97-50a798a0e4f6
2,3,3,6,0,0,038554734X,9780385547345,Doubleday,A scientist and single mother living in Califo...,0.00,LESSONS IN CHEMISTRY,Bonnie Garmus,by Bonnie Garmus,,https://storage.googleapis.com/du-prd/books/im...,328,500,https://www.amazon.com/dp/038554734X?tag=NYTBS...,,,,,,"[{'isbn10': '038554734X', 'isbn13': '978038554...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/717e0ead-9782-567d-af27-d9a6cfb1856f
3,4,4,54,0,0,1538724731,9781538724736,Grand Central,Lowen Ashleigh is hired by the husband of an i...,0.00,VERITY,Colleen Hoover,by Colleen Hoover,,https://storage.googleapis.com/du-prd/books/im...,324,500,https://www.amazon.com/dp/1791392792?tag=NYTBS...,,,,,,"[{'isbn10': '1791392792', 'isbn13': '978179139...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/013c7729-a1de-5e5c-921a-da43243f9a4a
4,5,5,9,0,0,0385548923,9780385548922,Doubleday,Two childhood friends follow in their fathers’...,0.00,THE BOYS FROM BILOXI,John Grisham,by John Grisham,,https://storage.googleapis.com/du-prd/books/im...,329,500,https://www.amazon.com/dp/0385548923?tag=NYTBS...,,,,,,"[{'isbn10': '0385548923', 'isbn13': '978038554...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/b59f42c0-d9d0-5535-bf4b-21e5d04d3a46


In [27]:
bs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   rank                  15 non-null     int64 
 1   rank_last_week        15 non-null     int64 
 2   weeks_on_list         15 non-null     int64 
 3   asterisk              15 non-null     int64 
 4   dagger                15 non-null     int64 
 5   primary_isbn10        15 non-null     object
 6   primary_isbn13        15 non-null     object
 7   publisher             15 non-null     object
 8   description           15 non-null     object
 9   price                 15 non-null     object
 10  title                 15 non-null     object
 11  author                15 non-null     object
 12  contributor           15 non-null     object
 13  contributor_note      15 non-null     object
 14  book_image            15 non-null     object
 15  book_image_width      15 non-null     int6

In [28]:
#write a function
def parse_books(book_list, date):
    '''
    Takes a response from NYT API and parses book into list of dictionaries
    '''

    books = []
    for i in book_list['results']['books']:
        dic = {}
        dic['age_group'] = i['age_group']
        dic['author'] = i['author'].encode("utf8")
        dic['book_review_link'] = i['book_review_link'].encode("utf8")
        dic['contributor'] = i['contributor'].encode("utf8")
        dic['dagger'] = i['dagger']
        dic['first_chapter_link'] = i['first_chapter_link'].encode("utf8")
        dic['price'] = i['price']
        dic['primary_isbn10'] = i['primary_isbn10'].encode("utf8")
        dic['primary_isbn13'] = i['primary_isbn13'].encode("utf8")
        dic['publisher'] = i['publisher'].encode("utf8")
        dic['sunday_review_link'] = i['sunday_review_link'].encode("utf8")
        dic['title'] = i['title'].encode("utf8")
        dic['weeks_on_list'] = i['weeks_on_list']
        dic['date'] = date  # add a date so that we know when it was a best-seller
        books.append(dic)
    return(books)  


In [29]:
# Check if it works- split in 2
BOOKS_ROOT = "https://api.nytimes.com/svc/books/v3/lists"
LIST = "combined-print-and-e-book-fiction"   # you can change to other lists available at https://developer.nytimes.com

# Add your API key here
API_KEY = keys.json

bestsellers = []
weeks = []
num_calls_to_make = 2  # Number of API calls to make
weeks_per_call = 260  # Number of weeks to fetch per API call

for call_num in range(num_calls_to_make):
    ctr = call_num * weeks_per_call
    start_date = datetime.date.today() - datetime.timedelta(weeks=ctr)
    end_date = start_date - datetime.timedelta(weeks=weeks_per_call)
    
    for i in range(weeks_per_call):
        curr_date = start_date - datetime.timedelta(weeks=i)
        print(curr_date)
        url = "%s/%s/%s?api-key=%s" % (BOOKS_ROOT, curr_date, LIST, API_KEY)
        url = url.strip()
        print(url)
        r = requests.get(url)
        
        if r.status_code == 200:
            results = r.json()
            books = json_normalize(results["results"]["books"])
            bestsellers.append(books)
            weeks.append(str(curr_date))
        else:
            print("API call did not work!")
            print(r)
        
        time.sleep(15)

   # Save or process the data for the current call here
    call_data = pd.concat(bestsellers)
    bestsellers1 = pd.concat([bestsellers, call_data])
    
    # Reset the lists for the next call
    bestsellers = []
    weeks = []

    # Wait for 24 hours before making the next call
    if call_num < num_calls_to_make - 1:
        time.sleep(24 * 60 * 60)

# Save or process the data for the final call here
final_data = pd.concat(bestsellers)
df = pd.concat([df, final_data])

# The dataframe 'df' now contains the combined data from all the API calls

2023-05-15
https://api.nytimes.com/svc/books/v3/lists/2023-05-15/combined-print-and-e-book-fiction?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb


C:\Users\melan\AppData\Local\Temp\ipykernel_3744\2623874419.py:28: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  books = json_normalize(results["results"]["books"])


2023-05-08
https://api.nytimes.com/svc/books/v3/lists/2023-05-08/combined-print-and-e-book-fiction?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb


C:\Users\melan\AppData\Local\Temp\ipykernel_3744\2623874419.py:28: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  books = json_normalize(results["results"]["books"])


2023-05-01
https://api.nytimes.com/svc/books/v3/lists/2023-05-01/combined-print-and-e-book-fiction?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb


...

2018-06-04
https://api.nytimes.com/svc/books/v3/lists/2018-06-04/combined-print-and-e-book-fiction?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb


C:\Users\melan\AppData\Local\Temp\ipykernel_3744\2623874419.py:28: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  books = json_normalize(results["results"]["books"])


2018-05-28
https://api.nytimes.com/svc/books/v3/lists/2018-05-28/combined-print-and-e-book-fiction?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb


C:\Users\melan\AppData\Local\Temp\ipykernel_3744\2623874419.py:28: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  books = json_normalize(results["results"]["books"])


NameError: name 'df' is not defined

In [35]:
df = pd.DataFrame(call_data)

In [38]:
df.head()

,rank,rank_last_week,weeks_on_list,asterisk,dagger,primary_isbn10,primary_isbn13,publisher,description,price,title,author,contributor,contributor_note,book_image,book_image_width,book_image_height,amazon_product_url,age_group,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,isbns,buy_links,book_uri
0,1,0,1,0,0,0316402982,9780316402989,"Little, Brown",The 23rd book in the Women’s Murder Club serie...,0.00,THE 23RD MIDNIGHT,James Patterson and Maxine Paetro,by James Patterson and Maxine Paetro,,https://storage.googleapis.com/du-prd/books/im...,326,500,https://www.amazon.com/dp/0316402788?tag=NYTBS...,,,,,,"[{'isbn10': '0316402788', 'isbn13': '978031640...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/8d902754-be73-5c91-8e58-3688d03d19d4
1,2,1,2,0,0,0593441273,9780593441275,Berkley,A former couple pretend to be together for the...,0.00,HAPPY PLACE,Emily Henry,by Emily Henry,,https://storage.googleapis.com/du-prd/books/im...,331,500,https://www.amazon.com/dp/0593441273?tag=NYTBS...,,,,,,"[{'isbn10': '0593441273', 'isbn13': '978059344...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/9ab3a59a-e572-5452-ae0c-eedeadb05831
2,3,0,1,0,0,0593438558,9780593438558,Berkley,"A decade after a one-day adventure, Will appea...",0.00,MEET ME AT THE LAKE,Carley Fortune,by Carley Fortune,,https://storage.googleapis.com/du-prd/books/im...,333,500,https://www.amazon.com/dp/0593438558?tag=NYTBS...,,,,,,"[{'isbn10': '0593438558', 'isbn13': '978059343...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/f6409475-c210-5baf-beda-d11fba9e9981
3,4,0,1,0,0,0802162177,9780802162175,Grove,Three generations of a family living on South ...,0.00,THE COVENANT OF WATER,Abraham Verghese,by Abraham Verghese,,https://storage.googleapis.com/du-prd/books/im...,320,500,https://www.amazon.com/dp/0802162177?tag=NYTBS...,,,,,,"[{'isbn10': '0802162177', 'isbn13': '978080216...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/5ebf1d34-4c68-560a-8330-7e65bedaaa1a
4,5,0,1,0,0,1649374046,9781649374042,Red Tower,Violet Sorrengail is urged by the commanding g...,0.00,FOURTH WING,Rebecca Yarros,by Rebecca Yarros,,https://storage.googleapis.com/du-prd/books/im...,309,500,https://www.amazon.com/dp/1649374046?tag=NYTBS...,,,,,,"[{'isbn10': '1649374046', 'isbn13': '978164937...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/106c2cf0-7d20-51b1-bad4-91c3ebcd131a


In [37]:
df.tail()

,rank,rank_last_week,weeks_on_list,asterisk,dagger,primary_isbn10,primary_isbn13,publisher,description,price,title,author,contributor,contributor_note,book_image,book_image_width,book_image_height,amazon_product_url,age_group,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,isbns,buy_links,book_uri
10,11,0,8,0,0,125013093X,9781250130938,St. Martin's,The story of a love triangle is told from seve...,0.00,THE WIFE BETWEEN US,Greer Hendricks and Sarah Pekkanen,by Greer Hendricks and Sarah Pekkanen,,https://storage.googleapis.com/du-prd/books/im...,326,495,https://www.amazon.com/Wife-Between-Us-Novel/d...,,,,,,"[{'isbn10': '1250130921', 'isbn13': '978125013...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/412f8d0a-dcb9-54d9-a381-ebd4d304dcb0
11,12,7,13,0,0,0735211221,9780735211223,Riverhead,In this psychological thriller by the author o...,0.00,INTO THE WATER,Paula Hawkins,by Paula Hawkins,,https://storage.googleapis.com/du-prd/books/im...,328,495,https://www.amazon.com/Into-Water-Novel-Paula-...,,,,,,"[{'isbn10': '0735211205', 'isbn13': '978073521...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/2d0f232c-cb7c-5837-a9d6-efe366eeb444
12,13,11,34,0,0,0425284697,9780425284698,Ballantine,A South Carolina lawyer learns about the quest...,0.00,BEFORE WE WERE YOURS,Lisa Wingate,by Lisa Wingate,,https://storage.googleapis.com/du-prd/books/im...,330,495,https://www.amazon.com/Before-We-Were-Yours-No...,,,,,,"[{'isbn10': '0425284689', 'isbn13': '978042528...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/1a6fb174-dd0c-564e-83b7-d00ec45d52cd
13,14,0,27,0,0,038549081X,9780385490818,Houghton Mifflin Harcourt,Men and women in a dystopian future. The basis...,0.00,THE HANDMAID'S TALE,Margaret Atwood,by Margaret Atwood,,https://storage.googleapis.com/du-prd/books/im...,330,485,https://www.amazon.com/Handmaids-Tale-Margaret...,,,,https://www.nytimes.com/1986/02/09/books/no-he...,,"[{'isbn10': '038549081X', 'isbn13': '978038549...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/0868766f-b674-50f9-a665-b1c5f0f17d13
14,15,15,2,0,0,0525505318,9780525505310,Penguin,A couple’s secrets emerge after their baby dis...,0.00,THE COUPLE NEXT DOOR,Shari Lapena,by Shari Lapena,,https://storage.googleapis.com/du-prd/books/im...,328,495,http://www.amazon.com/Couple-Next-Door-Novel/d...,,,,,,"[{'isbn10': '0735221081', 'isbn13': '978073522...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/d8986982-6ff0-5bb3-8837-2157887fd9a9


In [40]:
df.shape

(3900, 26)

In [41]:
len(weeks)

260

In [ ]:
'''###check if it works
BOOKS_ROOT = "https://api.nytimes.com/svc/books/v3/lists"
LIST = "combined-print-and-e-book-fiction"   #you can change to other lists available at https://developer.nytimes.com

# Add your API key here
API_KEY = keys.json

bestsellers = []
weeks = []
num_calls_to_make = 520 # you can use up to 500 calls a day, so need to split
ctr = 0

# lists update once a week, every Sunday
for i in range(0,num_calls_to_make):
    today = datetime.datetime.today()
    idx = (today.weekday() + 1) % 7
    last_sun = today - datetime.timedelta((7*ctr)+idx)
    last_sun = last_sun.date()
    print(last_sun)
    url = "%s/%s/%s?api-key=%s" % (BOOKS_ROOT, last_sun, LIST, API_KEY)
    url = url.strip()
    print(url)
    r = requests.get(url)
    if (r.status_code == 200):
        results = r.json()
        books = json_normalize(results["results"]["books"]) #parse_books(results, date)
        bestsellers.append(books)
        weeks.append(str(last_sun))
    else:
        print("API call did not work!")
        print(r)
        #break
    ctr = ctr + 1
    time.sleep(12)

    #Splitting into 2 sections
    bestsellers_part1 = bestsellers[:260] #First 5 years of data, 260 weeks
    weeks_part1 = weeks[:260]
'''

In [43]:
#repeat weeks 16 times to add to df
#https://www.pythonforbeginners.com/lists/repeat-each-element-in-a-list-in-python
k = 15
repeated_weeks = []
for element in weeks:
    for i in range(k):
       repeated_weeks.append(element)

In [44]:
len(repeated_weeks)

3900

In [45]:
df.to_csv("C:\\Users\\melan\\Documents\\DA8\\Projects\\capstone\\last_five_years_nyt_bestsellers_data.csv", index=False)

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3900 entries, 0 to 14
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   rank                  3900 non-null   int64 
 1   rank_last_week        3900 non-null   int64 
 2   weeks_on_list         3900 non-null   int64 
 3   asterisk              3900 non-null   int64 
 4   dagger                3900 non-null   int64 
 5   primary_isbn10        3900 non-null   object
 6   primary_isbn13        3900 non-null   object
 7   publisher             3900 non-null   object
 8   description           3900 non-null   object
 9   price                 3900 non-null   object
 10  title                 3900 non-null   object
 11  author                3900 non-null   object
 12  contributor           3900 non-null   object
 13  contributor_note      3899 non-null   object
 14  book_image            3900 non-null   object
 15  book_image_width      3900 non-null   in

In [48]:
#rename df for clarity
combined_bestsellers = df

In [50]:
#after looking at head and tail, it looks like they switched from top 20 to top 15 at some point
#for normalization, let's narrow it down to top 15 across all weeks
#https://www.geeksforgeeks.org/python-pandas-dataframe-isin/
combined_bestsellers['rank'] = combined_bestsellers['rank'].astype(str)
filter_15 = combined_bestsellers['rank'].isin(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'])

In [52]:

top_15_combined_bestsellers = combined_bestsellers[filter_15]

In [53]:

top_15_combined_bestsellers.head()

,rank,rank_last_week,weeks_on_list,asterisk,dagger,primary_isbn10,primary_isbn13,publisher,description,price,title,author,contributor,contributor_note,book_image,book_image_width,book_image_height,amazon_product_url,age_group,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,isbns,buy_links,book_uri
0,1,0,1,0,0,0316402982,9780316402989,"Little, Brown",The 23rd book in the Women’s Murder Club serie...,0.00,THE 23RD MIDNIGHT,James Patterson and Maxine Paetro,by James Patterson and Maxine Paetro,,https://storage.googleapis.com/du-prd/books/im...,326,500,https://www.amazon.com/dp/0316402788?tag=NYTBS...,,,,,,"[{'isbn10': '0316402788', 'isbn13': '978031640...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/8d902754-be73-5c91-8e58-3688d03d19d4
1,2,1,2,0,0,0593441273,9780593441275,Berkley,A former couple pretend to be together for the...,0.00,HAPPY PLACE,Emily Henry,by Emily Henry,,https://storage.googleapis.com/du-prd/books/im...,331,500,https://www.amazon.com/dp/0593441273?tag=NYTBS...,,,,,,"[{'isbn10': '0593441273', 'isbn13': '978059344...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/9ab3a59a-e572-5452-ae0c-eedeadb05831
2,3,0,1,0,0,0593438558,9780593438558,Berkley,"A decade after a one-day adventure, Will appea...",0.00,MEET ME AT THE LAKE,Carley Fortune,by Carley Fortune,,https://storage.googleapis.com/du-prd/books/im...,333,500,https://www.amazon.com/dp/0593438558?tag=NYTBS...,,,,,,"[{'isbn10': '0593438558', 'isbn13': '978059343...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/f6409475-c210-5baf-beda-d11fba9e9981
3,4,0,1,0,0,0802162177,9780802162175,Grove,Three generations of a family living on South ...,0.00,THE COVENANT OF WATER,Abraham Verghese,by Abraham Verghese,,https://storage.googleapis.com/du-prd/books/im...,320,500,https://www.amazon.com/dp/0802162177?tag=NYTBS...,,,,,,"[{'isbn10': '0802162177', 'isbn13': '978080216...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/5ebf1d34-4c68-560a-8330-7e65bedaaa1a
4,5,0,1,0,0,1649374046,9781649374042,Red Tower,Violet Sorrengail is urged by the commanding g...,0.00,FOURTH WING,Rebecca Yarros,by Rebecca Yarros,,https://storage.googleapis.com/du-prd/books/im...,309,500,https://www.amazon.com/dp/1649374046?tag=NYTBS...,,,,,,"[{'isbn10': '1649374046', 'isbn13': '978164937...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/106c2cf0-7d20-51b1-bad4-91c3ebcd131a


In [54]:
len(top_15_combined_bestsellers)

3900

In [55]:
len(repeated_weeks)

3900

In [56]:
#add a week column
top_15_combined_bestsellers['week'] = repeated_weeks

#and check to make sure it worked correctly
top_15_combined_bestsellers.tail(20)

,rank,rank_last_week,weeks_on_list,asterisk,dagger,primary_isbn10,primary_isbn13,publisher,description,price,title,author,contributor,contributor_note,book_image,book_image_width,book_image_height,amazon_product_url,age_group,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,isbns,buy_links,book_uri,week
10,11,7,5,0,0,073521736X,9780735217362,Putnam,The 28th book in the Prey series. A federal ma...,0.00,TWISTED PREY,John Sandford,by John Sandford,,https://storage.googleapis.com/du-prd/books/im...,328,495,https://www.amazon.com/Twisted-Prey-Novel-John...,,,,,,"[{'isbn10': '0735217351', 'isbn13': '978073521...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/67c1d531-ba20-5c69-b97a-fbaa5d39554b,2018-06-04
11,12,15,3,0,0,0525505318,9780525505310,Penguin,A couple’s secrets emerge after their baby dis...,0.00,THE COUPLE NEXT DOOR,Shari Lapena,by Shari Lapena,,https://storage.googleapis.com/du-prd/books/im...,328,495,http://www.amazon.com/Couple-Next-Door-Novel/d...,,,,,,"[{'isbn10': '0735221081', 'isbn13': '978073522...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/d8986982-6ff0-5bb3-8837-2157887fd9a9,2018-06-04
12,13,10,16,0,0,125016561X,9781250165619,St. Martin's,A former prisoner of war returns from Vietnam ...,0.00,THE GREAT ALONE,Kristin Hannah,by Kristin Hannah,,https://storage.googleapis.com/du-prd/books/im...,326,495,https://www.amazon.com/Great-Alone-Novel-Krist...,,,,,,"[{'isbn10': '0312577230', 'isbn13': '978031257...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/92bd289c-529f-5558-8c5e-d0247e1351da,2018-06-04
13,14,3,2,0,0,0062390821,9780062390820,Morrow,Two families are brought together when the dau...,0.00,BY INVITATION ONLY,Dorothea Benton Frank,by Dorothea Benton Frank,,https://storage.googleapis.com/du-prd/books/im...,330,495,https://www.amazon.com/Invitation-Only-Dorothe...,,,,,,"[{'isbn10': '0062390821', 'isbn13': '978006239...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/bf535c81-82ce-5674-8793-1fe5870f1f05,2018-06-04
14,15,8,3,0,0,1250126096,9781250126092,St. Martin's,An eccentric millionaire enlists the attorney ...,0.00,THE HIGH TIDE CLUB,Mary Kay Andrews,by Mary Kay Andrews,,https://storage.googleapis.com/du-prd/books/im...,326,495,https://www.amazon.com/High-Tide-Club-Novel-eb...,,,,,,"[{'isbn10': '1250126061', 'isbn13': '978125012...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/25d190d1-c796-5f10-a3b9-75fc47340868,2018-06-04
0,1,0,1,0,0,1101884045,9781101884041,Delacorte,A magazine columnist meets an array of Hollywo...,0.00,THE CAST,Danielle Steel,by Danielle Steel,,https://storage.googleapis.com/du-prd/books/im...,326,495,https://www.amazon.com/Cast-Novel-Danielle-Ste...,,,,,,"[{'isbn10': '1101884037', 'isbn13': '978110188...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/a8a0fa07-896a-5b5a-a572-f1e72e1b88f1,2018-05-28
1,2,1,3,0,0,0316274046,9780316274043,"Little, Brown",The latest installment in the Women’s Murder C...,0.00,THE 17TH SUSPECT,James Patterson and Maxine Paetro,by James Patterson and Maxine Paetro,,https://storage.googleapis.com/du-prd/books/im...,320,495,https://www.amazon.com/17th-Suspect-Womens-Mur...,,,,,,"[{'isbn10': '0316274046', 'isbn13': '978031627...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/7ae807ae-5910-5fe4-a92a-d634dbac645b,2018-05-28
2,3,0,1,0,0,006239083X,9780062390837,Morrow,Two families are brought together when the dau...,0.00,BY INVITATION ONLY,Dorothea Benton Frank,by Dorothea Benton Frank,,https://storage.googleapis.com/du-prd/books/im...,330,495,https://www.amazon.com/Invitation-Only-Dorothe...,,,,,,"[{'isbn10': '0062390821', 'isbn13': '978006239...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/ff8d046e-b560-580f-8f88-0c7bdd18fb59,2018-05-28
3,4,0,1,0,0,1538714450,9781538714454,Grand Central,Princess Caroline calls on Jack Morgan when he...,0.00,PRINCESS,James Patterson and Rees Jones,by James Patter

In [57]:
#and a column indicating which list these come from
k2 = 3900
weekly_fiction_list = ['Combined Print & E-Book Fiction']
nyt_list_weekly_fiction = []

for element in weekly_fiction_list:
    for i in range(k2):
        nyt_list_weekly_fiction.append(element)

len(nyt_list_weekly_fiction)

3900

In [58]:
#add nyt_list as a column
top_15_combined_bestsellers['nyt_list'] = nyt_list_weekly_fiction

#and check to make sure it worked correctly
top_15_combined_bestsellers.tail(20)

,rank,rank_last_week,weeks_on_list,asterisk,dagger,primary_isbn10,primary_isbn13,publisher,description,price,title,author,contributor,contributor_note,book_image,book_image_width,book_image_height,amazon_product_url,age_group,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,isbns,buy_links,book_uri,week,nyt_list
10,11,7,5,0,0,073521736X,9780735217362,Putnam,The 28th book in the Prey series. A federal ma...,0.00,TWISTED PREY,John Sandford,by John Sandford,,https://storage.googleapis.com/du-prd/books/im...,328,495,https://www.amazon.com/Twisted-Prey-Novel-John...,,,,,,"[{'isbn10': '0735217351', 'isbn13': '978073521...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/67c1d531-ba20-5c69-b97a-fbaa5d39554b,2018-06-04,Combined Print & E-Book Fiction
11,12,15,3,0,0,0525505318,9780525505310,Penguin,A couple’s secrets emerge after their baby dis...,0.00,THE COUPLE NEXT DOOR,Shari Lapena,by Shari Lapena,,https://storage.googleapis.com/du-prd/books/im...,328,495,http://www.amazon.com/Couple-Next-Door-Novel/d...,,,,,,"[{'isbn10': '0735221081', 'isbn13': '978073522...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/d8986982-6ff0-5bb3-8837-2157887fd9a9,2018-06-04,Combined Print & E-Book Fiction
12,13,10,16,0,0,125016561X,9781250165619,St. Martin's,A former prisoner of war returns from Vietnam ...,0.00,THE GREAT ALONE,Kristin Hannah,by Kristin Hannah,,https://storage.googleapis.com/du-prd/books/im...,326,495,https://www.amazon.com/Great-Alone-Novel-Krist...,,,,,,"[{'isbn10': '0312577230', 'isbn13': '978031257...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/92bd289c-529f-5558-8c5e-d0247e1351da,2018-06-04,Combined Print & E-Book Fiction
13,14,3,2,0,0,0062390821,9780062390820,Morrow,Two families are brought together when the dau...,0.00,BY INVITATION ONLY,Dorothea Benton Frank,by Dorothea Benton Frank,,https://storage.googleapis.com/du-prd/books/im...,330,495,https://www.amazon.com/Invitation-Only-Dorothe...,,,,,,"[{'isbn10': '0062390821', 'isbn13': '978006239...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/bf535c81-82ce-5674-8793-1fe5870f1f05,2018-06-04,Combined Print & E-Book Fiction
14,15,8,3,0,0,1250126096,9781250126092,St. Martin's,An eccentric millionaire enlists the attorney ...,0.00,THE HIGH TIDE CLUB,Mary Kay Andrews,by Mary Kay Andrews,,https://storage.googleapis.com/du-prd/books/im...,326,495,https://www.amazon.com/High-Tide-Club-Novel-eb...,,,,,,"[{'isbn10': '1250126061', 'isbn13': '978125012...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/25d190d1-c796-5f10-a3b9-75fc47340868,2018-06-04,Combined Print & E-Book Fiction
0,1,0,1,0,0,1101884045,9781101884041,Delacorte,A magazine columnist meets an array of Hollywo...,0.00,THE CAST,Danielle Steel,by Danielle Steel,,https://storage.googleapis.com/du-prd/books/im...,326,495,https://www.amazon.com/Cast-Novel-Danielle-Ste...,,,,,,"[{'isbn10': '1101884037', 'isbn13': '978110188...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/a8a0fa07-896a-5b5a-a572-f1e72e1b88f1,2018-05-28,Combined Print & E-Book Fiction
1,2,1,3,0,0,0316274046,9780316274043,"Little, Brown",The latest installment in the Women’s Murder C...,0.00,THE 17TH SUSPECT,James Patterson and Maxine Paetro,by James Patterson and Maxine Paetro,,https://storage.googleapis.com/du-prd/books/im...,320,495,https://www.amazon.com/17th-Suspect-Womens-Mur...,,,,,,"[{'isbn10': '0316274046', 'isbn13': '978031627...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/7ae807ae-5910-5fe4-a92a-d634dbac645b,2018-05-28,Combined Print & E-Book Fiction
2,3,0,1,0,0,006239083X,9780062390837,Morrow,Two families are brought together when the dau...,0.00,BY INVITATION ONLY,Dorothea Benton Frank,by Dorothea Benton Frank,,https://storage.googleapis.com/du-prd/books/im...,330,495,https://www.amazon.com/Invitation-Only-Dorothe...,,,,,,"[{'isbn10': '0062390821', 'isbn13': '978006239...","[{'name': 'Amazon', 'url': 'https://ww

In [62]:
#save to another csv
top_15_combined_bestsellers.to_csv("C:\\Users\\melan\\Documents\\DA8\\Projects\\capstone\\data\\top_15_fiction_bestsellers_since_may_28_2018.csv", index=False)

In [4]:
#get previous 5 years

BOOKS_ROOT = "https://api.nytimes.com/svc/books/v3/lists"
LIST = "combined-print-and-e-book-fiction"
API_KEY = keys.json

bestsellers = []
weeks = []
num_calls_to_make = 1
weeks_per_call = 260  # Number of weeks to fetch per API call

for call_num in range(num_calls_to_make):
    ctr = call_num * weeks_per_call
    start_date = datetime.date(2018, 5, 21) - datetime.timedelta(weeks=ctr)
    end_date = start_date - datetime.timedelta(weeks=weeks_per_call)
    
    for i in range(weeks_per_call):
        curr_date = start_date - datetime.timedelta(weeks=i)
        print(curr_date)
        url = "%s/%s/%s?api-key=%s" % (BOOKS_ROOT, curr_date, LIST, API_KEY)
        url = url.strip()
        print(url)
        r = requests.get(url)
        
        if r.status_code == 200:
            results = r.json()
            books = json_normalize(results["results"]["books"])
            bestsellers.append(books)
            weeks.append(str(curr_date))
        else:
            print("API call did not work!")
            print(r)
        
        time.sleep(15)

#Save or process the data for the current call here
    call_data = pd.concat(bestsellers)
    bestsellers2 = pd.concat([bestsellers, call_data])

2018-05-21
https://api.nytimes.com/svc/books/v3/lists/2018-05-21/combined-print-and-e-book-fiction?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb


C:\Users\melan\AppData\Local\Temp\ipykernel_22472\373377676.py:27: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  books = json_normalize(results["results"]["books"])


2018-05-14
https://api.nytimes.com/svc/books/v3/lists/2018-05-14/combined-print-and-e-book-fiction?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb


C:\Users\melan\AppData\Local\Temp\ipykernel_22472\373377676.py:27: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  books = json_normalize(results["results"]["books"])


2018-05-07
https://api.nytimes.com/svc/books/v3/lists/2018-05-07/combined-print-and-e-book-fiction?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb


...

2013-06-10
https://api.nytimes.com/svc/books/v3/lists/2013-06-10/combined-print-and-e-book-fiction?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb


C:\Users\melan\AppData\Local\Temp\ipykernel_22472\373377676.py:27: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  books = json_normalize(results["results"]["books"])


2013-06-03
https://api.nytimes.com/svc/books/v3/lists/2013-06-03/combined-print-and-e-book-fiction?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb


C:\Users\melan\AppData\Local\Temp\ipykernel_22472\373377676.py:27: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  books = json_normalize(results["results"]["books"])


TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid

In [16]:
df = pd.DataFrame(call_data)

In [17]:
df.head()

,rank,rank_last_week,weeks_on_list,asterisk,dagger,primary_isbn10,primary_isbn13,publisher,description,price,title,author,contributor,contributor_note,book_image,book_image_width,book_image_height,amazon_product_url,age_group,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,isbns,buy_links,book_uri
0,1,1,2,0,0,0316274046,9780316274043,"Little, Brown",The latest installment in the Women’s Murder C...,0.00,THE 17TH SUSPECT,James Patterson and Maxine Paetro,by James Patterson and Maxine Paetro,,https://storage.googleapis.com/du-prd/books/im...,320.0,495.0,https://www.amazon.com/17th-Suspect-Womens-Mur...,,,,,,"[{'isbn10': '0316274046', 'isbn13': '978031627...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/7ae807ae-5910-5fe4-a92a-d634dbac645b
1,2,2,4,0,0,1538761394,9781538761397,Grand Central,"Amos Decker, known as the Memory Man, puts his...",0.00,THE FALLEN,David Baldacci,by David Baldacci,,https://storage.googleapis.com/du-prd/books/im...,328.0,495.0,https://www.amazon.com/Fallen-Memory-Man-David...,,,,,,"[{'isbn10': '1538761394', 'isbn13': '978153876...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/1125e0ae-7b0e-51da-b838-2cf6b360150c
2,3,0,1,0,0,0525483446,9780525483441,Bantam,The rogue F.B.I. agent Jane Hawk is on the lam...,0.00,THE CROOKED STAIRCASE,Dean Koontz,by Dean Koontz,,https://storage.googleapis.com/du-prd/books/im...,326.0,495.0,https://www.amazon.com/Crooked-Staircase-Jane-...,,,,,,"[{'isbn10': '052548342X', 'isbn13': '978052548...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/2f7e77b5-ac10-5162-8e7d-0a26253b870c
3,4,0,1,0,0,1250126096,9781250126092,St. Martin's,An eccentric millionaire enlists the attorney ...,0.00,THE HIGH TIDE CLUB,Mary Kay Andrews,by Mary Kay Andrews,,https://storage.googleapis.com/du-prd/books/im...,326.0,495.0,https://www.amazon.com/High-Tide-Club-Novel-eb...,,,,,,"[{'isbn10': '1250126061', 'isbn13': '978125012...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/25d190d1-c796-5f10-a3b9-75fc47340868
4,5,3,3,0,0,073521736X,9780735217362,Putnam,The 28th book in the Prey series. A federal ma...,0.00,TWISTED PREY,John Sandford,by John Sandford,,https://storage.googleapis.com/du-prd/books/im...,328.0,495.0,https://www.amazon.com/Twisted-Prey-Novel-John...,,,,,,"[{'isbn10': '0735217351', 'isbn13': '978073521...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/67c1d531-ba20-5c69-b97a-fbaa5d39554b


In [18]:
df.tail()

,rank,rank_last_week,weeks_on_list,asterisk,dagger,primary_isbn10,primary_isbn13,publisher,description,price,title,author,contributor,contributor_note,book_image,book_image_width,book_image_height,amazon_product_url,age_group,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,isbns,buy_links,book_uri
15,16,0,0,0,0,0345507843,9780345507846,Ballantine,"Over the course of four decades, five diverse ...",0.00,THE WEDNESDAY SISTERS,Meg Waite Clayton,by Meg Waite Clayton,,https://storage.googleapis.com/du-prd/books/im...,326.0,495.0,http://www.amazon.com/The-Wednesday-Sisters-A-...,,,,,,"[{'isbn10': '0345507843', 'isbn13': '978034550...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/6c9b59fc-cbac-5e9d-865d-df59de7097db
16,17,0,0,0,0,None,A00B00CLXL4HW,Cassia Leo,Claire must choose between the surfer Adam Par...,0.00,PIECES OF YOU,Cassia Leo,by Cassia Leo,,https://storage.googleapis.com/du-prd/books/im...,330.0,495.0,http://www.amazon.com/Pieces-You-Shattered-Hea...,,,,,,[],"[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/dbc5ef95-5f1d-5009-a9af-8fa91912a54f
17,18,0,0,0,0,None,9780062292117,Morrow,All good intentions go out the window when the...,0.00,STEALING HARPER,Molly McAdams,by Molly McAdams,,https://storage.googleapis.com/du-prd/books/im...,308.0,495.0,http://www.amazon.com/Stealing-Harper-Taking-C...,,,,,,[],"[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/d2fa42c7-11be-5b73-aa68-7bac5b752827
18,19,0,0,0,0,1455546313,9781455546312,Grand Central,The ordeal of a New Orleans woman suspected of...,0.00,FRENCH SILK,Sandra Brown,by Sandra Brown,,https://storage.googleapis.com/du-prd/books/im...,128.0,209.0,http://www.amazon.com/French-Silk-Sandra-Brown...,,,,,,"[{'isbn10': '1455546313', 'isbn13': '978145554...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/fcce016d-ec30-5991-bdbf-4493a3b97780
19,20,0,0,0,0,0553897845,9780553897845,Bantam,In the frozen wastes to the north of Winterfel...,0.00,A GAME OF THRONES,George RR Martin,by George R.R. Martin,,https://storage.googleapis.com/du-prd/books/im...,301.0,495.0,http://www.amazon.com/Game-Thrones-Song-Fire-B...,,,,,https://www.nytimes.com/2011/04/10/arts/televi...,"[{'isbn10': '0553573403', 'isbn13': '978055357...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/4dfc5533-8154-5fdd-ae02-efee1f59dd3d


In [19]:
df.shape

(4855, 26)

In [20]:
len(weeks)

260

In [21]:
#repeat weeks 16 times to add to df
#https://www.pythonforbeginners.com/lists/repeat-each-element-in-a-list-in-python
k = 15
repeated_weeks = []
for element in weeks:
    for i in range(k):
       repeated_weeks.append(element)

In [22]:
len(repeated_weeks)

3900

In [23]:
df.to_csv("C:\\Users\\melan\\Documents\\DA8\\Projects\\capstone\\2013-2018_nyt_bestsellers_data.csv", index=False)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4855 entries, 0 to 19
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   rank                  4855 non-null   int64  
 1   rank_last_week        4855 non-null   int64  
 2   weeks_on_list         4855 non-null   int64  
 3   asterisk              4855 non-null   int64  
 4   dagger                4855 non-null   int64  
 5   primary_isbn10        4855 non-null   object 
 6   primary_isbn13        4855 non-null   object 
 7   publisher             4855 non-null   object 
 8   description           4855 non-null   object 
 9   price                 4855 non-null   object 
 10  title                 4855 non-null   object 
 11  author                4855 non-null   object 
 12  contributor           4855 non-null   object 
 13  contributor_note      4855 non-null   object 
 14  book_image            4846 non-null   object 
 15  book_image_width      4

In [25]:
#after looking at head and tail, it looks like they switched from top 20 to top 15 at some point
#for normalization, let's narrow it down to top 15 across all weeks
#https://www.geeksforgeeks.org/python-pandas-dataframe-isin/
df['rank'] = df['rank'].astype(str)
filter_15 = df['rank'].isin(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'])

In [26]:
top_15_combined_bestsellers_2013_2018 = df[filter_15]

In [27]:

top_15_combined_bestsellers_2013_2018.head()

,rank,rank_last_week,weeks_on_list,asterisk,dagger,primary_isbn10,primary_isbn13,publisher,description,price,title,author,contributor,contributor_note,book_image,book_image_width,book_image_height,amazon_product_url,age_group,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,isbns,buy_links,book_uri
0,1,1,2,0,0,0316274046,9780316274043,"Little, Brown",The latest installment in the Women’s Murder C...,0.00,THE 17TH SUSPECT,James Patterson and Maxine Paetro,by James Patterson and Maxine Paetro,,https://storage.googleapis.com/du-prd/books/im...,320.0,495.0,https://www.amazon.com/17th-Suspect-Womens-Mur...,,,,,,"[{'isbn10': '0316274046', 'isbn13': '978031627...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/7ae807ae-5910-5fe4-a92a-d634dbac645b
1,2,2,4,0,0,1538761394,9781538761397,Grand Central,"Amos Decker, known as the Memory Man, puts his...",0.00,THE FALLEN,David Baldacci,by David Baldacci,,https://storage.googleapis.com/du-prd/books/im...,328.0,495.0,https://www.amazon.com/Fallen-Memory-Man-David...,,,,,,"[{'isbn10': '1538761394', 'isbn13': '978153876...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/1125e0ae-7b0e-51da-b838-2cf6b360150c
2,3,0,1,0,0,0525483446,9780525483441,Bantam,The rogue F.B.I. agent Jane Hawk is on the lam...,0.00,THE CROOKED STAIRCASE,Dean Koontz,by Dean Koontz,,https://storage.googleapis.com/du-prd/books/im...,326.0,495.0,https://www.amazon.com/Crooked-Staircase-Jane-...,,,,,,"[{'isbn10': '052548342X', 'isbn13': '978052548...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/2f7e77b5-ac10-5162-8e7d-0a26253b870c
3,4,0,1,0,0,1250126096,9781250126092,St. Martin's,An eccentric millionaire enlists the attorney ...,0.00,THE HIGH TIDE CLUB,Mary Kay Andrews,by Mary Kay Andrews,,https://storage.googleapis.com/du-prd/books/im...,326.0,495.0,https://www.amazon.com/High-Tide-Club-Novel-eb...,,,,,,"[{'isbn10': '1250126061', 'isbn13': '978125012...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/25d190d1-c796-5f10-a3b9-75fc47340868
4,5,3,3,0,0,073521736X,9780735217362,Putnam,The 28th book in the Prey series. A federal ma...,0.00,TWISTED PREY,John Sandford,by John Sandford,,https://storage.googleapis.com/du-prd/books/im...,328.0,495.0,https://www.amazon.com/Twisted-Prey-Novel-John...,,,,,,"[{'isbn10': '0735217351', 'isbn13': '978073521...","[{'name': 'Amazon', 'url': 'https://www.amazon...",nyt://book/67c1d531-ba20-5c69-b97a-fbaa5d39554b


In [28]:
len(top_15_combined_bestsellers_2013_2018)

3900

In [29]:
len(repeated_weeks)

3900

In [31]:
#add a week column
top_15_combined_bestsellers_2013_2018['week'] = repeated_weeks

#and check to make sure it worked correctly
top_15_combined_bestsellers_2013_2018.tail(40)

C:\Users\melan\AppData\Local\Temp\ipykernel_22472\18292709.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_15_combined_bestsellers_2013_2018['week'] = repeated_weeks


,rank,rank_last_week,weeks_on_list,asterisk,dagger,primary_isbn10,primary_isbn13,publisher,description,price,title,author,contributor,contributor_note,book_image,book_image_width,book_image_height,amazon_product_url,age_group,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,isbns,buy_links,book_uri,week
5,6,7,3,0,0,0307351939,9780307351937,Broadway,An “oral history” of an imagined Zombie War th...,0.00,WORLD WAR Z,Max Brooks,by Max Brooks,,https://storage.googleapis.com/du-prd/books/im...,322.0,495.0,http://www.amazon.com/World-Market-Movie-Tie-I...,,,,,,"[{'isbn10': '0307346617', 'isbn13': '978030734...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/c9d0451b-cfa0-56a9-a79c-ec911c031aff,2013-06-17
6,7,3,7,0,0,1455521280,9781455521289,Grand Central,The government hitman Will Robie uncovers a se...,0.00,THE HIT,David Baldacci,by David Baldacci,,https://storage.googleapis.com/du-prd/books/im...,128.0,194.0,http://www.amazon.com/The-Will-Robie-David-Bal...,,,,,,"[{'isbn10': '1455521213', 'isbn13': '978145552...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/e60c9940-e444-5d21-b0f3-d63a70643c75,2013-06-17
7,8,0,37,0,0,0553897845,9780553897845,Bantam,In the frozen wastes to the north of Winterfel...,0.00,A GAME OF THRONES,George RR Martin,by George R.R. Martin,,https://storage.googleapis.com/du-prd/books/im...,301.0,495.0,http://www.amazon.com/Game-Thrones-Song-Fire-B...,,,,,https://www.nytimes.com/2011/04/10/arts/televi...,"[{'isbn10': '0553573403', 'isbn13': '978055357...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/4dfc5533-8154-5fdd-ae02-efee1f59dd3d,2013-06-17
8,9,0,1,0,0,1455517070,9781455517077,Grand Central,The quadriplegic forensics expert Lincoln Rhym...,0.00,THE KILL ROOM,Jeffery Deaver,by Jeffery Deaver,,https://storage.googleapis.com/du-prd/books/im...,330.0,494.0,http://www.amazon.com/The-Kill-Room-Lincoln-Rh...,,,,,,"[{'isbn10': '1455517062', 'isbn13': '978145551...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/a730ec12-f587-5621-8f84-7e04b8634420,2013-06-17
9,10,11,2,0,0,1628470356,9781628470352,Georgia Cates,Will Jack Henry’s absence leave Laurelyn wanti...,0.00,BEAUTY FROM SURRENDER,Georgia Cates,by Georgia Cates,,https://storage.googleapis.com/du-prd/books/im...,330.0,495.0,http://www.amazon.com/Beauty-Surrender-Series-...,,,,,,"[{'isbn10': '1628470356', 'isbn13': '978162847...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/62ad3808-6d1a-5fbc-966d-07850b77d36b,2013-06-17
10,11,9,5,0,0,None,9781101621196,Putnam,Lucas Davenport is drawn into the investigatio...,0.00,SILKEN PREY,John Sandford,by John Sandford,,https://storage.googleapis.com/du-prd/books/im...,280.0,495.0,http://www.amazon.com/Silken-Prey-Lucas-Davenp...,,,,,,"[{'isbn10': '0399159312', 'isbn13': '978039915...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/e68fb3cb-cb49-5b23-9490-d5ddb2003560,2013-06-17
11,12,0,1,0,0,,9781250019653,St. Martin's,"After separating from her husband, a lifestyle...",0.00,LADIES' NIGHT,Mary Kay Andrews,by Mary Kay Andrews,,https://storage.googleapis.com/du-prd/books/im...,328.0,495.0,http://www.amazon.com/Ladies-Night-Mary-Kay-An...,,,,,,"[{'isbn10': '1250019672', 'isbn13': '978125001...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/24465f54-7de3-5f31-980b-fc50b720e61a,2013-06-17
12,13,0,52,0,0,0307588386,9780307588388,Crown,A woman disappears on the day of her fifth ann...,0.00,GONE GIRL,Gillian Flynn,by Gillian Flynn,,https://storage.googleapis.com/du-prd/books/im...,136.0,200.0,http://www.amazon.com/Gone-Girl-Novel-Gillian-...,,https://www.nytimes.com/2012/05/30/books/gone-...,,https://www.nytimes.com/2012/06/17/books/revie...,,"[{'isbn10': '030758836X', 'isbn13': '978030758...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/d76ecc9a-b7ad-5cf0-9ef5-cde9a3eba381,2013-06-17
13,14,10,6,0,0,,9780316210843,"Little, Brown","A week after giving birth, Detective Lin

In [32]:
#and a column indicating which list these come from
k2 = 3900
weekly_fiction_list = ['Combined Print & E-Book Fiction']
nyt_list_weekly_fiction = []

for element in weekly_fiction_list:
    for i in range(k2):
        nyt_list_weekly_fiction.append(element)

len(nyt_list_weekly_fiction)

3900

In [33]:
#add nyt_list as a column
top_15_combined_bestsellers_2013_2018['nyt_list'] = nyt_list_weekly_fiction

#and check to make sure it worked correctly
top_15_combined_bestsellers_2013_2018.tail(20)

C:\Users\melan\AppData\Local\Temp\ipykernel_22472\94425587.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_15_combined_bestsellers_2013_2018['nyt_list'] = nyt_list_weekly_fiction


,rank,rank_last_week,weeks_on_list,asterisk,dagger,primary_isbn10,primary_isbn13,publisher,description,price,title,author,contributor,contributor_note,book_image,book_image_width,book_image_height,amazon_product_url,age_group,book_review_link,first_chapter_link,sunday_review_link,article_chapter_link,isbns,buy_links,book_uri,week,nyt_list
10,11,0,1,0,0,1628470356,9781628470352,Georgia Cates,Will Jack Henry’s absence leave Laurelyn wanti...,0.00,BEAUTY FROM SURRENDER,Georgia Cates,by Georgia Cates,,https://storage.googleapis.com/du-prd/books/im...,330.0,495.0,http://www.amazon.com/Beauty-Surrender-Series-...,,,,,,"[{'isbn10': '1628470356', 'isbn13': '978162847...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/62ad3808-6d1a-5fbc-966d-07850b77d36b,2013-06-10,Combined Print & E-Book Fiction
11,12,0,1,0,0,None,9781101623848,Jove,"A killer is on the loose in New Orleans, and a...",0.00,LEOPARD'S PREY,Christine Feehan,by Christine Feehan,,https://storage.googleapis.com/du-prd/books/im...,307.0,495.0,http://www.amazon.com/Leopards-Prey-A-Leopard-...,,,,,,"[{'isbn10': '0515151556', 'isbn13': '978051515...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/cf138bf9-80d0-5d7f-aa21-7b449f1d682b,2013-06-10,Combined Print & E-Book Fiction
12,13,0,1,0,0,None,A00B00C2GEJT8,Amy Harmon,"A young British teacher takes a tough, trouble...",0.00,A DIFFERENT BLUE,Amy Harmon,by Amy Harmon,,https://storage.googleapis.com/du-prd/books/im...,330.0,495.0,http://www.amazon.com/Different-Blue-Amy-Harmo...,,,,,,[],"[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/1a6fa774-484c-598c-80b6-bca343cc4930,2013-06-10,Combined Print & E-Book Fiction
13,14,8,7,0,0,None,9781101621219,Putnam,A former criminal attorney who has been acquit...,0.00,WHISKEY BEACH,Nora Roberts,by Nora Roberts,,https://storage.googleapis.com/du-prd/books/im...,128.0,194.0,http://www.amazon.com/Whiskey-Beach-Nora-Rober...,,,,,,"[{'isbn10': '0399159894', 'isbn13': '978039915...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/81a6f29a-7b95-523d-b90b-83c1f35b770b,2013-06-10,Combined Print & E-Book Fiction
14,15,10,4,0,0,142996393X,9781429963930,Tor/Tom Doherty,Government agencies breed child geniuses and t...,0.00,ENDER'S GAME,Orson Scott Card,by Orson Scott Card,,https://storage.googleapis.com/du-prd/books/im...,291.0,475.0,http://www.amazon.com/Enders-Game-The-Ender-Qu...,,,,,,"[{'isbn10': '0812550706', 'isbn13': '978081255...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/1568ec4d-b638-5fab-8e15-f045d409c44e,2013-06-10,Combined Print & E-Book Fiction
0,1,1,2,0,0,0385537867,9780385537865,Doubleday,"The symbologist Robert Langdon, on the run in ...",0.00,INFERNO,Dan Brown,by Dan Brown,,https://storage.googleapis.com/du-prd/books/im...,276.0,495.0,http://www.amazon.com/Inferno-Robert-Langdon-D...,,https://www.nytimes.com/2013/05/13/books/infer...,,,,"[{'isbn10': '0385537859', 'isbn13': '978038553...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/a986073a-5877-5e90-8efb-94e6e17521bc,2013-06-03,Combined Print & E-Book Fiction
1,2,0,1,0,0,159463176X,9781594631764,Riverhead,A multigenerational family saga centers on a b...,0.00,AND THE MOUNTAINS ECHOED,Khaled Hosseini,by Khaled Hosseini,,https://storage.googleapis.com/du-prd/books/im...,317.0,495.0,http://www.amazon.com/And-Mountains-Echoed-Kha...,,https://www.nytimes.com/2013/05/21/books/and-t...,,,,"[{'isbn10': '159463176X', 'isbn13': '978159463...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt://book/bb6a383e-9d83-5024-92a1-1df3e6ae81c6,2013-06-03,Combined Print & E-Book Fiction
2,3,2,5,0,0,1455521280,9781455521289,Grand Central,The government hitman Will Robie uncovers a se...,0.00,THE HIT,David Baldacci,by David Baldacci,,https://storage.googleapis.com/du-prd/books/im...,128.0,194.0,http://www.amazon.com/The-Will-Robie-David-Bal...,,,,,,"[{'isbn10': '1455521213', 'isbn13': '978145552...","[{'name': 'Amazon', 'url': 'http://www.amazon....",nyt

In [34]:
#save to another csv
top_15_combined_bestsellers_2013_2018.to_csv("C:\\Users\\melan\\Documents\\DA8\\Projects\\capstone\\data\\top_15_fiction_bestsellers_2013_to_2018.csv", index=False)